<a href="https://colab.research.google.com/github/Pratiksha100/AnalyticsProject/blob/cheewan/pyconcorde_upgrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%%capture
%pip install git+https://github.com/jvkersch/pyconcorde

In [12]:
# install tweak file
%cp '/content/drive/MyDrive/Analytics Project/Concorde/concorde_class.py' '/content'
%cp '/content/drive/MyDrive/Analytics Project/Concorde/test.py' '/content'
%cp '/content/drive/MyDrive/Analytics Project/Concorde/concorde_to_replace/tsp.py' '/usr/local/lib/python3.10/dist-packages/concorde'
%cp '/content/drive/MyDrive/Analytics Project/Concorde/concorde_to_replace/util.py' '/usr/local/lib/python3.10/dist-packages/concorde'

In [ ]:
'''
%%capture
!cp "/content/drive/MyDrive/Analytics Project/Concorde.zip" "/content"
!unzip Concorde.zip
!bash /content/Concorde/install_concorde.sh
'''

In [13]:
import numpy as np
import pandas as pd
import math

In [ ]:
Warmsen_01_distance = pd.read_csv('/content/drive/MyDrive/Analytics Project/AOP_DP_Analytics/Data/Instances/Warmsen/distances/distances_31600-01.dat' ,
                                  names=['col1', 'col2', 'col3']
                                  )
# distance_index = Warmsen_01_distance['col1'].value_counts().index
check = Warmsen_01_distance.set_index(['col1', 'col2'])['col3'].unstack().values

In [1]:
import pandas as pd
import os
import numpy as np


def create_dist_mat_from_atsp(path_of_file): #input : path of the .atsp file
    # specify the path to the .atsp file
    file_path = path_of_file
    print(file_path)


    # initialize variables
    n_nodes = None
    dist_matrix = []

    # open and read the file
    with open(file_path) as f:
        for line in f:
            # extract the number of nodes
            if "DIMENSION" in line:
                n_nodes = int(line.split(":")[1].strip())
            # extract the distance matrix
            elif "EDGE_WEIGHT_SECTION" in line:
                while True:
                    line = f.readline()
                    if "EOF" in line:
                        break
                    dist_matrix.append(list(map(int, line.split())))
            # continue reading until end of file
            elif "EOF" in line:
                break


    #resizing the distance matrix to a square matrix
    list_of_lists = dist_matrix

    flat_list = []
    for sublist in list_of_lists:
        for item in sublist:
            flat_list.append(item)


    # Example 1D list
    lst = flat_list

    # Calculate the size of the square matrix
    n = len(lst)
    size = int(np.sqrt(n))

    # Reshape the 1D list into a 2D numpy array with the desired shape
    arr = np.reshape(lst, (size, size))

    # Print the resulting array
    dist_mat = np.array(arr)

    print("No of cities : "  , n_nodes)
    print(dist_mat.shape)
    return dist_mat #returns the distance matrix

In [2]:
%%time
from concorde_class import Concorde

distances = create_dist_mat_from_atsp('/content/drive/MyDrive/Analytics Project/TSPLIB_ATSP/rbg443.atsp')

concorde = Concorde()

concorde_sol = concorde.run_ProbATSP(distances)
print('-------------------------------------------------------------------------')
print(concorde.route)
print('-------------------------------------------------------------------------')
print(concorde.optimal_value)

/content/drive/MyDrive/Analytics Project/TSPLIB_ATSP/rbg443.atsp
No of cities :  443
(443, 443)
-------------------------------------------------------------------------
[0, 413, 431, 207, 327, 212, 235, 31, 70, 252, 350, 337, 125, 126, 179, 405, 204, 113, 369, 156, 153, 303, 304, 440, 408, 234, 297, 139, 128, 19, 276, 294, 217, 141, 129, 34, 198, 177, 215, 224, 436, 263, 326, 27, 59, 183, 249, 32, 1, 284, 135, 178, 273, 360, 49, 200, 218, 109, 438, 421, 36, 110, 112, 434, 144, 400, 236, 248, 305, 40, 230, 435, 53, 149, 9, 130, 299, 338, 223, 246, 136, 286, 76, 150, 426, 389, 406, 51, 380, 401, 107, 173, 361, 225, 394, 292, 387, 316, 278, 341, 148, 176, 202, 233, 293, 266, 75, 390, 417, 222, 370, 11, 255, 264, 347, 90, 21, 58, 33, 30, 302, 105, 93, 241, 291, 355, 397, 88, 2, 13, 22, 23, 157, 56, 312, 335, 328, 319, 377, 91, 43, 412, 368, 374, 106, 282, 187, 424, 425, 429, 116, 216, 26, 81, 147, 42, 133, 256, 265, 442, 239, 124, 164, 340, 122, 97, 142, 64, 66, 52, 372, 38, 25, 65, 6, 27